In [70]:
import numpy as np
import pandas as pd
from scipy.sparse import csgraph
import os

import sys
sys.path.append('../')
from src import data
from scipy.stats import spearmanr

# https://github.com/aestrivex/bctpy
# dokumentace asi jenom pro matlab? https://sites.google.com/site/bctnet/
# from bct.algorithms import efficiency

from netneurotools import metrics # zjistit, proč tohle nefunguje přes pip 
# (mám to stažené lokálně z Githubu a to pak funguje)

from nilearn import plotting

# https://neuraldatascience.io/7-eeg/mne_data.html
%matplotlib widget

PLOT=False

!date -I

2024-01-29


In [71]:
# load data F-TRACT
response_probability = np.loadtxt('../data/external/F-TRACT/DKT/probability.txt.gz')
response_amplitude = np.loadtxt('../data/external/F-TRACT/DKT/amplitude__median.txt.gz')

dkt_roi = ['ctx-lh-bankssts','ctx-lh-caudalanteriorcingulate','ctx-lh-caudalmiddlefrontal','ctx-lh-corpuscallosum','ctx-lh-cuneus','ctx-lh-entorhinal','ctx-lh-fusiform','ctx-lh-inferiorparietal','ctx-lh-inferiortemporal','ctx-lh-isthmuscingulate','ctx-lh-lateraloccipital','ctx-lh-lateralorbitofrontal','ctx-lh-lingual','ctx-lh-medialorbitofrontal','ctx-lh-middletemporal','ctx-lh-parahippocampal','ctx-lh-paracentral','ctx-lh-parsopercularis','ctx-lh-parsorbitalis','ctx-lh-parstriangularis','ctx-lh-pericalcarine','ctx-lh-postcentral','ctx-lh-posteriorcingulate','ctx-lh-precentral','ctx-lh-precuneus','ctx-lh-rostralanteriorcingulate','ctx-lh-rostralmiddlefrontal','ctx-lh-superiorfrontal','ctx-lh-superiorparietal','ctx-lh-superiortemporal','ctx-lh-supramarginal','ctx-lh-frontalpole','ctx-lh-temporalpole','ctx-lh-transversetemporal','ctx-lh-insula','ctx-rh-bankssts','ctx-rh-caudalanteriorcingulate','ctx-rh-caudalmiddlefrontal','ctx-rh-corpuscallosum','ctx-rh-cuneus','ctx-rh-entorhinal','ctx-rh-fusiform','ctx-rh-inferiorparietal','ctx-rh-inferiortemporal','ctx-rh-isthmuscingulate','ctx-rh-lateraloccipital','ctx-rh-lateralorbitofrontal','ctx-rh-lingual','ctx-rh-medialorbitofrontal','ctx-rh-middletemporal','ctx-rh-parahippocampal','ctx-rh-paracentral','ctx-rh-parsopercularis','ctx-rh-parsorbitalis','ctx-rh-parstriangularis','ctx-rh-pericalcarine','ctx-rh-postcentral','ctx-rh-posteriorcingulate','ctx-rh-precentral','ctx-rh-precuneus','ctx-rh-rostralanteriorcingulate','ctx-rh-rostralmiddlefrontal','ctx-rh-superiorfrontal','ctx-rh-superiorparietal','ctx-rh-superiortemporal','ctx-rh-supramarginal','ctx-rh-frontalpole','ctx-rh-temporalpole','ctx-rh-transversetemporal','ctx-rh-insula']

with open('../data/external/F-TRACT/DKT/DKT.txt', 'r') as f:
    parcell_ids =  [i for i,line in enumerate(f) if line[:-1] in dkt_roi]

# select ROI
response_probability = response_probability[np.ix_(parcell_ids, parcell_ids)]
response_amplitude  = response_amplitude[np.ix_(parcell_ids, parcell_ids)]

print(response_probability.shape)

(70, 70)


In [72]:
rootdir = "../data/external/hcp/DKT/"
scdir = "1StructuralConnectivity/"
fcdir = "2FunctionalConnectivity/"

W, L = np.zeros((70,70)), np.zeros((70,70))

for i in range(200):
    counts_file = rootdir+scdir+f"{i:03d}/Counts.csv"
    with open(counts_file,"r") as cf:
        c = np.genfromtxt(cf,delimiter=" ")
    W += c/200
    
    lenghts_file = rootdir+scdir+f"{i:03d}/Lengths.csv"
    with open(lenghts_file,"r") as lf:
        l = np.genfromtxt(lf,delimiter=" ")
    L += l/200


In [73]:
# shortcut for correlation of X with response probability and amplitude 
def coor_with_response(X):
    r_probability = spearmanr(X.flatten(),response_probability.flatten(),nan_policy='omit')
    r_amplitude_median = spearmanr(X.flatten(),response_amplitude.flatten(),nan_policy='omit')

    print(f"Correlation with response probabilty: {r_probability.statistic:.3f} (p={r_probability.pvalue:.3f})")
    print(f"Correlation with response amplitude: {r_amplitude_median.statistic:.3f} (p={r_amplitude_median.pvalue:.3f})")

In [74]:
coor_with_response(W) 

Correlation with response probabilty: 0.461 (p=0.000)
Correlation with response amplitude: -0.004 (p=0.953)


In [75]:
coor_with_response(L) 

Correlation with response probabilty: -0.574 (p=0.000)
Correlation with response amplitude: -0.116 (p=0.085)


In [76]:
# shortest path efficiency
shorest_paths,_ = metrics.distance_wei_floyd(L)
shorest_path_eff = np.divide(1,shorest_paths)

coor_with_response(shorest_path_eff)

Correlation with response probabilty: 0.452 (p=0.000)
Correlation with response amplitude: -0.003 (p=0.968)


/tmp/ipykernel_24085/1122485199.py:3: RuntimeWarning: divide by zero encountered in divide
  shorest_path_eff = np.divide(1,shorest_paths)


In [77]:
com = metrics.communicability_wei(W)

coor_with_response(com)

Correlation with response probabilty: 0.329 (p=0.000)
Correlation with response amplitude: -0.186 (p=0.006)


In [78]:
SI = metrics.search_information(W,L)

coor_with_response(SI)

Correlation with response probabilty: -0.406 (p=0.000)
Correlation with response amplitude: 0.065 (p=0.385)


In [79]:
# https://netneurolab.github.io/netneurotools/_modules/netneurotools/metrics.html#diffusion_efficiency
diffusion = metrics.diffusion_efficiency(W)[1] # tahle funkce určitě má dostat SC, a ne délky L

coor_with_response(diffusion)

Correlation with response probabilty: -0.233 (p=0.000)
Correlation with response amplitude: -0.089 (p=0.186)


/home/katterrina/fi_muni/0_diplomka/__venv__/lib/python3.10/site-packages/netneurotools/metrics.py:780: RuntimeWarning: divide by zero encountered in divide
  E_diff = np.divide(1, mfpt)


In [80]:
# tady té funkci možná dávám špatný vstup?
_,_,nav_paths,_,_ = metrics.navigation_wu(L, W)
nav_eff = np.divide(1,nav_paths)

coor_with_response(nav_eff)

/tmp/ipykernel_24085/2471498993.py:3: RuntimeWarning: divide by zero encountered in divide
  nav_eff = np.divide(1,nav_paths)


Correlation with response probabilty: 0.455 (p=0.000)
Correlation with response amplitude: -0.008 (p=0.918)
